In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display, HTML

print("🎯 СОЗДАНИЕ ДАШБОРДА В JUPYTER NOTEBOOK")
print("="*50)

# 1. ЗАГРУЗКА ДАННЫХ
print("📥 Загружаю данные...")
try:
    rides = pd.read_csv('data/processing_rides.csv', parse_dates=['start_date', 'end_date'])
    print(f"✅ Загружено {len(rides)} поездок")
except:
    print("❌ Ошибка загрузки")
    # Создаем тестовые данные
    np.random.seed(42)
    dates = pd.date_range('2023-04-01', '2023-04-30', freq='H')
    rides = pd.DataFrame({
        'id': range(1000),
        'start_date': np.random.choice(dates, 1000),
        'distance': np.random.uniform(1000, 10000, 1000),
        'duration': np.random.uniform(300, 3600, 1000),
        'start_district': np.random.choice(['Северный', 'Южный', 'Западный', 'Восточный'], 1000),
        'promo': np.random.choice([0, 1], 1000, p=[0.7, 0.3])
    })
    print(f"Создано тестовых данных: {len(rides)} поездок")

# 2. ПОДГОТОВКА ДАННЫХ
print("🔧 Подготавливаю данные...")
rides['duration_min'] = rides['duration'] / 60
rides['distance_km'] = rides['distance'] / 1000
rides['hour'] = rides['start_date'].dt.hour
rides['weekday'] = rides['start_date'].dt.day_name()

print("Данные готовы")

# 3. KPI КАРТОЧКИ
print("\nКЛЮЧЕВЫЕ ПОКАЗАТЕЛИ:")
print("-"*30)

total = len(rides)
avg_duration = rides['duration_min'].mean()
avg_distance = rides['distance_km'].mean()
promo_pct = (rides['promo'].sum() / total * 100)

kpi_html = f"""
<div style="display: flex; gap: 10px; margin: 20px 0;">
    <div style="flex: 1; background: #3498db; color: white; padding: 15px; border-radius: 10px; text-align: center;">
        <div style="font-size: 24px;">🚗</div>
        <div style="font-size: 12px;">Всего поездок</div>
        <div style="font-size: 24px; font-weight: bold;">{total:,}</div>
    </div>
    <div style="flex: 1; background: #2ecc71; color: white; padding: 15px; border-radius: 10px; text-align: center;">
        <div style="font-size: 24px;">⏱️</div>
        <div style="font-size: 12px;">Ср. длительность</div>
        <div style="font-size: 24px; font-weight: bold;">{avg_duration:.1f} мин</div>
    </div>
    <div style="flex: 1; background: #e74c3c; color: white; padding: 15px; border-radius: 10px; text-align: center;">
        <div style="font-size: 24px;">📏</div>
        <div style="font-size: 12px;">Ср. дистанция</div>
        <div style="font-size: 24px; font-weight: bold;">{avg_distance:.1f} км</div>
    </div>
    <div style="flex: 1; background: #f39c12; color: white; padding: 15px; border-radius: 10px; text-align: center;">
        <div style="font-size: 24px;">🎁</div>
        <div style="font-size: 12px;">Поездки по промо</div>
        <div style="font-size: 24px; font-weight: bold;">{promo_pct:.1f}%</div>
    </div>
</div>
"""

display(HTML(kpi_html))

# 4. ГРАФИК 1: ПО ДНЯМ НЕДЕЛИ
print("\nГРАФИК 1: РАСПРЕДЕЛЕНИЕ ПО ДНЯМ НЕДЕЛИ")

weekday_counts = rides['weekday'].value_counts().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
]).reset_index()
weekday_counts.columns = ['День', 'Количество']

fig1 = px.bar(
    weekday_counts,
    x='День',
    y='Количество',
    color='Количество',
    color_continuous_scale='Blues',
    title='Количество поездок по дням недели'
)
fig1.show()

# 5. ГРАФИК 2: ПО ЧАСАМ
print("\nГРАФИК 2: РАСПРЕДЕЛЕНИЕ ПО ЧАСАМ")

hourly_counts = rides['hour'].value_counts().sort_index().reset_index()
hourly_counts.columns = ['Час', 'Количество']

fig2 = px.bar(
    hourly_counts,
    x='Час',
    y='Количество',
    title='Количество поездок по часам суток',
    color='Количество',
    color_continuous_scale='Greens'
)
fig2.update_xaxes(dtick=1)
fig2.show()

# 6. ГРАФИК 3: ТОП РАЙОНОВ
print("\nГРАФИК 3: ТОП РАЙОНОВ")

district_counts = rides['start_district'].value_counts().head(10).reset_index()
district_counts.columns = ['Район', 'Количество']

fig3 = px.bar(
    district_counts,
    y='Район',
    x='Количество',
    orientation='h',
    title='Топ-10 районов по количеству поездок',
    color='Количество',
    color_continuous_scale='Reds'
)
fig3.show()

# 7. ГРАФИК 4: ДИСТАНЦИЯ VS ДЛИТЕЛЬНОСТЬ
print("\nГРАФИК 4: СООТНОШЕНИЕ ПАРАМЕТРОВ")

fig4 = px.scatter(
    rides.sample(min(500, len(rides))),
    x='distance_km',
    y='duration_min',
    color='start_district',
    size='distance_km',
    hover_data=['start_date'],
    title='Соотношение дистанции и длительности поездок',
    labels={'distance_km': 'Дистанция (км)', 'duration_min': 'Длительность (мин)'}
)
fig4.show()

# 8. ТАБЛИЦА С ДАННЫМИ
print("\nТАБЛИЦА: ПОСЛЕДНИЕ 10 ПОЕЗДОК")

# Берем последние поездки
recent_rides = rides.sort_values('start_date', ascending=False).head(10).copy()
recent_rides['start_date'] = recent_rides['start_date'].dt.strftime('%d.%m.%Y %H:%M')
recent_rides['promo'] = recent_rides['promo'].apply(lambda x: '✅ Да' if x == 1 else '❌ Нет')

# Отображаем таблицу
display(recent_rides[['start_date', 'start_district', 'distance_km', 'duration_min', 'promo']]
        .style
        .set_caption('Последние 10 поездок')
        .background_gradient(subset=['distance_km', 'duration_min'], cmap='YlOrRd')
        .format({'distance_km': '{:.1f} км', 'duration_min': '{:.1f} мин'}))



🎯 СОЗДАНИЕ ДАШБОРДА В JUPYTER NOTEBOOK
📥 Загружаю данные...
✅ Загружено 97354 поездок
🔧 Подготавливаю данные...
Данные готовы

КЛЮЧЕВЫЕ ПОКАЗАТЕЛИ:
------------------------------



ГРАФИК 1: РАСПРЕДЕЛЕНИЕ ПО ДНЯМ НЕДЕЛИ



ГРАФИК 2: РАСПРЕДЕЛЕНИЕ ПО ЧАСАМ



ГРАФИК 3: ТОП РАЙОНОВ



ГРАФИК 4: СООТНОШЕНИЕ ПАРАМЕТРОВ



ТАБЛИЦА: ПОСЛЕДНИЕ 10 ПОЕЗДОК


,start_date,start_district,distance_km,duration_min,promo
97353,31.07.2023 23:57,Юго-Восточный,3.6 км,22.0 мин,❌ Нет
97352,31.07.2023 23:51,Северо-Западный,6.5 км,46.4 мин,❌ Нет
97351,31.07.2023 23:48,Ленинский,2.8 км,21.3 мин,❌ Нет
97350,31.07.2023 23:46,Ленинский,5.0 км,39.6 мин,❌ Нет
97349,31.07.2023 23:45,Юго-Восточный,3.3 км,21.8 мин,❌ Нет
97348,31.07.2023 23:43,Октябрьский,4.8 км,58.4 мин,❌ Нет
97347,31.07.2023 23:43,Северо-Западный,4.5 км,27.2 мин,❌ Нет
97346,31.07.2023 23:42,Северо-Западный,4.6 км,34.6 мин,❌ Нет
97345,31.07.2023 23:41,Центральный,4.3 км,27.1 мин,❌ Нет
97344,31.07.2023 23:36,Центральный,2.8 км,19.4 мин,❌ Нет
